# Business Consultant: Retail Operations Pipeline
Objective: Automate the ingestion, cleaning, and strategic analysis of transaction data for a high-volume cafe.

Datasheet: https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training/data

This dataset is released under the CC BY-SA 4.0 License. 

In [5]:
%%capture
!pip install kagglehub pandas matplotlib seaborn

import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import os

In [34]:
## Cafe Analytics class. 
# All the analytics function will be defined in this class.

class CafeAnalytics:
    def __init__(self, dataset_handle):
        self.handle = dataset_handle
        self.df = None
        self.path = None

    def ingest_data(self):
        print(f"--- Ingesting data from: {self.handle} ---")
        self.path = kagglehub.dataset_download(self.handle)
        
        # Dynamic search for the CSV
        csv_files = [f for f in os.listdir(self.path) if f.endswith('.csv')]
        full_path = os.path.join(self.path, csv_files[0])
        self.df = pd.read_csv(full_path)
        print(f"Success! Loaded {len(self.df)} records.")

    def audit_data(self):
        # A quick report for the 'Client'
        print(f"\n--- Business Data Audit ---")
        print(f"Missing Values: {self.df.isnull().sum().sum()}")
        print(f"Unique Products: {self.df['Item'].nunique()}")
        display(self.df.describe())

    def pre_clean_audit(self):
        if self.df is None:
            print("Error: Load data first!")
            return

        # 1. Financial Calculations
        avg_spend = pd.to_numeric(self.df['Total Spent'], errors='coerce').mean()
        
        # Identify the 'Junk' (Standardizing to lower case just for the check)
        
        # 1. Standardize the text (handles spaces and casing)
        clean_items = self.df['Item'].astype(str).str.strip().str.upper()

        # 2. Check for the "Keywords" AND check for "Empty Strings"
        mask_junk = (clean_items.isin(['UNKNOWN', 'ERROR', 'NAN', 'NONE', ''])) | (clean_items == "")
        junk_df = self.df[mask_junk]
        
        junk_total_spent = pd.to_numeric(junk_df['Total Spent'], errors='coerce')
        lost_revenue = junk_total_spent.sum()

        mask_nulls = self.df['Item'].isna()
        nulls_df = self.df[mask_nulls]

        nulls_total_spent = pd.to_numeric(nulls_df['Total Spent'], errors='coerce')
        null_lost_revenue = nulls_total_spent.sum()

        total_rows = len(self.df)
        junk_rows_count = len(junk_df)
        junk_percent = (junk_rows_count / total_rows) * 100

        # 2. The Consultant Report
        print("======= PRE-CLEANING STRATEGIC AUDIT =======")
        print(f"Total Transactions Audited: {total_rows:,}")
        print(f"Average Transaction Value: ${avg_spend:.2f}")
        print("-" * 44)
        print(f"CRITICAL ALERT: 'Dark Data' detected.")
        print(f"Unidentified Items (UNKNOWN/ERROR): {junk_rows_count} rows")
        
        print(f"Data Corruption Rate: {junk_percent:.2f}%")
        print(f"Revenue at Risk: ${lost_revenue:,.2f}")
        print(f"Revenue at Risk from empty item name: ${null_lost_revenue:,.2f}")
        print("-" * 44)
        print("Observation: This revenue is currently untraceable to specific ")
        print("products, preventing accurate inventory and marketing decisions.")
        print("============================================")
        
        # Store these stats if you want to use them for the AI later
        self.pre_clean_stats = {
            "lost_revenue": lost_revenue,
            "junk_percent": junk_percent
        }


    def clean_data(self):
        print("--- Starting Data Sanitization ---")
        
        # 1. Capture original size to report how much "junk" we removed
        initial_rows = len(self.df)
        
        # 2. Standardize Categories (Fixes 'coffee' vs 'Coffee ')
        if 'Item' in self.df.columns:
            self.df['Item'] = self.df['Item'].str.strip().str.title()
        
        # 3. Filter out "Invalid" entries (The 'Error' and 'Unknown' items)
        # Consultants remove data that would "pollute" the final business strategy
        self.df = self.df[~self.df['Item'].isin(['Error', 'Unknown'])]
        
        # 4. Handle Missing Prices (Imputation)
        # Instead of deleting, we fill missing prices with the Median (the 'Safe' middle value)
        if 'Price' in self.df.columns:
            median_price = self.df['Price'].median()
            self.df['Price'] = self.df['Price'].fillna(median_price)
            
        # Fill missing prices based on what the item actually is
        #self.df['Price'] = self.df['Price'].fillna(self.df.groupby('Item')['Price'].transform('median'))


        # 5. Final Report for the Client
        final_rows = len(self.df)
        removed = initial_rows - final_rows
        print(f"Sanitization Complete: Removed {removed} invalid rows.")
        print(f"Data Health: 100% of remaining {final_rows} rows are standardized.")




# Input Phase

This dataset is intentionally "dirty," with missing values, inconsistent data, and errors introduced to provide a realistic scenario for data cleaning and exploratory data analysis (EDA). It can be used to practice cleaning techniques, data wrangling, and feature engineering.

In [39]:
# Initialize
dataset_handle = 'ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training'
consultant_tool = CafeAnalytics(dataset_handle)

# Run Ingestion
consultant_tool.ingest_data()

# Run pre clean audit
consultant_tool.pre_clean_audit()

# Run Audit
consultant_tool.audit_data()

# Set normalize=True to get decimals (0.60 = 60%)
print(f'{consultant_tool.df['Item'].value_counts(normalize=True) * 100}')

#consultant_tool.df.head()


--- Ingesting data from: ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training ---
Success! Loaded 10000 records.
======= PRE-CLEANING STRATEGIC AUDIT =======
Total Transactions Audited: 10,000
Average Transaction Value: $8.92
--------------------------------------------
CRITICAL ALERT: 'Dark Data' detected.
Unidentified Items (UNKNOWN/ERROR): 969 rows
Data Corruption Rate: 9.69%
Revenue at Risk: $8,140.00
Revenue at Risk from empty item name: $2,929.00
--------------------------------------------
Observation: This revenue is currently untraceable to specific 
products, preventing accurate inventory and marketing decisions.

--- Business Data Audit ---
Missing Values: 6826
Unique Products: 10


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_9226047,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


Item
Juice       12.113375
Coffee      12.051309
Salad       11.875453
Cake        11.782352
Sandwich    11.699597
Smoothie    11.337540
Cookie      11.296162
Tea         11.265129
UNKNOWN      3.558498
ERROR        3.020585
Name: proportion, dtype: float64


Item might contain missing or invalid values  
Quantity might contain missing or invalid values  
Price per unit might contain missing or invalid values  
Payment method might contain missing or invalid values  
Location might contain missing or invalid values  
Transaction date might contain missing or invalid values  

Payment and Location have a lot of null values (more than 20%)  

# Cleaning Phase

In [40]:
consultant_tool.clean_data()

print(consultant_tool.df.isnull().sum())

--- Starting Data Sanitization ---
Sanitization Complete: Removed 636 invalid rows.
Data Health: 100% of remaining 9364 rows are standardized.
Transaction ID         0
Item                 333
Quantity             124
Price Per Unit       165
Total Spent          158
Payment Method      2412
Location            3049
Transaction Date     149
dtype: int64


# Transformation / Engineering (Logic Phase)

# Reporting / AI (Output Phase)